In [1]:
import re
import subprocess
import os
import json


In [2]:
path = "/Users/user/apps-files/github/cyber_completitions/text-files/"
os.chdir(path)


In [3]:
parsed_commands = []

chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
]


In [4]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append("-h")

    _out = subprocess.run(_args, capture_output=True)
    _text = _out.stdout.decode("utf-8") + _out.stderr.decode("utf-8")

    # with open(command + ".txt", 'w') as file:
    #     file.write(_text)

    # # file_content = f"### {command}\n\n```\n{_text}\n```\n\n"

    # with open("all_commands2.md", 'a') as file:
    #     file.write(file_content)

    _lines = _text.split("\n")
    return _lines, command


_lines, command = get_cli_output("cyber")
_lines


['Bostrom Bootloader Hub',
 '',
 'Usage:',
 '  cyber [command]',
 '',
 'Available Commands:',
 '  add-genesis-account Add a genesis account to genesis.json',
 '  collect-gentxs      Collect genesis txs and output a genesis.json file',
 '  config              Create or query an application CLI configuration file',
 '  debug               Tool for helping with debugging your application',
 '  export              Export state to JSON',
 '  gentx               Generate a genesis tx carrying a self delegation',
 '  help                Help about any command',
 '  init                Initialize private validator, p2p, genesis, and application configuration files',
 "  keys                Manage your application's keys",
 '  migrate             Migrate genesis to a specified target version',
 '  query               Querying subcommands',
 '  rollback            rollback cosmos-sdk and tendermint state by one height',
 '  start               Run the full node',
 '  status              Query re

In [5]:
get_cli_output("cyber query evidence")


(["Error: invalid evidence hash: encoding/hex: invalid byte: U+002D '-'",
  'Usage:',
  '  cyber query evidence [flags]',
  '',
  'Flags:',
  '      --count-total       count total number of records in evidence to query for',
  '      --height int        Use a specific height to query state at (this can error if the node is pruning state)',
  '  -h, --help              help for evidence',
  '      --limit uint        pagination limit of evidence to query for (default 100)',
  '      --node string       <host>:<port> to Tendermint RPC interface for this chain (default "tcp://localhost:26657")',
  '      --offset uint       pagination offset of evidence to query for',
  '  -o, --output string     Output format (text|json) (default "text")',
  '      --page uint         pagination page of evidence to query for. This sets offset to a multiple of limit (default 1)',
  '      --page-key string   pagination page-key of evidence to query for',
  '      --reverse           results are sorted in

In [6]:
def parse_chapters(_lines: list, command):

    chapters = {}
    chapters["command"] = command
    chapter = "Title"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


chapters = parse_chapters(_lines, command)
chapters


{'command': 'cyber',
 'Title': ['Bostrom Bootloader Hub'],
 'Usage': ['  cyber [command]'],
 'Available Commands': ['  add-genesis-account Add a genesis account to genesis.json',
  '  collect-gentxs      Collect genesis txs and output a genesis.json file',
  '  config              Create or query an application CLI configuration file',
  '  debug               Tool for helping with debugging your application',
  '  export              Export state to JSON',
  '  gentx               Generate a genesis tx carrying a self delegation',
  '  help                Help about any command',
  '  init                Initialize private validator, p2p, genesis, and application configuration files',
  "  keys                Manage your application's keys",
  '  migrate             Migrate genesis to a specified target version',
  '  query               Querying subcommands',
  '  rollback            rollback cosmos-sdk and tendermint state by one height',
  '  start               Run the full node',

In [7]:
def parse_avail_commands(chapters):
    _temp_commands = []

    if "Available Commands" in chapters:

        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                _temp_commands.append(_match.group(1))

        chapters["sub_commands"] = _temp_commands
        del chapters["Available Commands"]
    return chapters


chapters = parse_avail_commands(chapters)
chapters


{'command': 'cyber',
 'Title': ['Bostrom Bootloader Hub'],
 'Usage': ['  cyber [command]'],
 'Flags': ['  -h, --help                help for cyber',
  '      --home string         directory for config and data (default "/Users/user/.cyber")',
  '      --log_format string   The logging format (json|plain) (default "plain")',
  '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
  '      --trace               print out full stack trace on errors',
  'Use "cyber [command] --help" for more information about a command.'],
 'sub_commands': ['add-genesis-account',
  'collect-gentxs',
  'config',
  'debug',
  'export',
  'gentx',
  'help',
  'init',
  'keys',
  'migrate',
  'query',
  'rollback',
  'start',
  'status',
  'tendermint',
  'testnet',
  'tx',
  'validate-genesis',
  'version']}

In [8]:
def query_and_parse(command):
    _lines, command = get_cli_output(command)
    commands_dict = parse_chapters(_lines, command)
    commands_dict = parse_avail_commands(commands_dict)
    return commands_dict


commands_dict_list = [query_and_parse("cyber")]
commands_dict_list


[{'command': 'cyber',
  'Title': ['Bostrom Bootloader Hub'],
  'Usage': ['  cyber [command]'],
  'Flags': ['  -h, --help                help for cyber',
   '      --home string         directory for config and data (default "/Users/user/.cyber")',
   '      --log_format string   The logging format (json|plain) (default "plain")',
   '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
   '      --trace               print out full stack trace on errors',
   'Use "cyber [command] --help" for more information about a command.'],
  'sub_commands': ['add-genesis-account',
   'collect-gentxs',
   'config',
   'debug',
   'export',
   'gentx',
   'help',
   'init',
   'keys',
   'migrate',
   'query',
   'rollback',
   'start',
   'status',
   'tendermint',
   'testnet',
   'tx',
   'validate-genesis',
   'version']}]

In [9]:
def get_new_commands(chapters: dict):
    new_commands = []
    if "sub_commands" in chapters:
        
        for i in chapters["sub_commands"]:
            command_candidate = " ".join((chapters["command"] + " " + i).split())
            if command_candidate not in parsed_commands:
                new_commands.append(command_candidate)
                parsed_commands.append(command_candidate)

    return new_commands



In [10]:
def loop_commands_list_for_new_comands(commands_dict_list):
    new_commands = []
    for c in commands_dict_list:
        t = get_new_commands(c)
        if t != []:
            new_commands.extend(t)

    return new_commands


In [11]:
all_commands_dict_list = current_dict_list = [query_and_parse('cyber')]

while True:
    new_commands = loop_commands_list_for_new_comands(current_dict_list)
    if new_commands == []:
        break
    current_dict_list = [query_and_parse(command) for command in new_commands]
    all_commands_dict_list.extend(current_dict_list)


In [12]:
len(all_commands_dict_list)

284

In [ ]:
all_commands_dict_list

In [ ]:
new_commands = []
for c in commands_dict_list:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands


In [ ]:
c1 = [query_and_parse(i) for i in new_commands]
c1


In [ ]:
new_commands = []
for c in c1:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands


In [ ]:
c2 = [query_and_parse(i) for i in new_commands]
c2


In [ ]:
new_commands = []
for c in c2:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands


In [ ]:
c3 = [query_and_parse(i) for i in new_commands]
c3


In [ ]:
new_commands = []
for c in c3:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands


In [ ]:
c4 = [query_and_parse(i) for i in new_commands]
c4


In [ ]:
new_commands = []
for c in c4:
    t = get_new_commands(c)
    if t != None:
        new_commands.extend(t)

new_commands


In [ ]:
all_commands = commands_dict_list + c1 + c2 + c3 + c4
all_commands


In [ ]:
# n = 0
# for i in all_commands:
#     if i['command'] == 'cyber query evidence':
#         print(n)

#     n += 1

# del all_commands[40]


In [ ]:
for i in all_commands[:5]:
    print(
        re.match(
            r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[\w-]+))\s+(?:<(?P<format>.*)>)?\s*(?P<description>.*)?",
            i["Flags"][0],
        ).group(0)
    )


In [ ]:
count = 0
for i in all_commands:

    if i["Flags"] != []:
        i["flags_parsed"] = []
        for b in i["Flags"]:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                i["flags_parsed"].append(
                    (
                        match1.group("long"),
                        match1.group("short"),
                        match1.group("format"),
                        match1.group("description"),
                    )
                )


In [ ]:
for c in all_commands:
    print(f"\n# {c['title']}\nextern '{c['command']}' [")
    for f in c["flags_parsed"]:
        if "." not in f[0]:
            print(f"\t--{f[0]}{'(-'+f[1]+')' if f[1] else ''}\t\t# {f[3]}")
    print("]")


In [ ]:
import pandas as pd


In [ ]:
df = pd.DataFrame(all_commands).sort_values("command", ascending=False)


In [ ]:
path = "/Users/user/apps-files/github/cyber_completitions/json-files/cyber-query distribution.json"

with open(path, "r") as f:
    content = f.read()


In [ ]:
import pandas as pd

pd.DataFrame.from_records(json.loads(content))


In [ ]:
_list = []

for _file in os.scandir(
    "/Users/user/apps-files/github/cyber_completitions/json-files/"
):
    with open(_file, "r") as f:
        content = f.read()
        _list.append(json.loads(content))


In [ ]:
[i for i in _list if i != []]


In [ ]:
df = pd.concat([pd.DataFrame.from_records(i) for i in _list if i != []])
df


In [ ]:
df["com1"] = "cyber " + df["_command_root"] + " " + df["_command"]
df


In [ ]:
_l2 = df.loc[:, ["_description", "com1"]].to_dict("records")
_l2


In [ ]:
for l in _l2:
    print(f'# {l["_description"]}\nextern "{l["com1"]}" [] \n')


In [ ]:
for i in json.loads(content):
    print("\t", i["_command"], "\t\t#", i["_description"])


In [ ]:
parsed_sorted = sorted(parsed_commands)
parsed_sorted.insert(0, "cyber")
parsed_sorted


In [ ]:
[get_cli_output(i) for i in parsed_sorted]
